## Prepare the data

### Setup the tokenizer

In [3]:
from transformers import AutoTokenizer

# Ensure the order
SPECIAL_TOKENS = ["<persuader>", "<persuadee>"]
MPATH = "microsoft/DialoGPT-medium"

tokenizer = AutoTokenizer.from_pretrained(MPATH)
tokenizer.add_special_tokens({'additional_special_tokens': SPECIAL_TOKENS})
tokenizer.pad_token = tokenizer.eos_token

Downloading: 100%|██████████| 26.0/26.0 [00:00<00:00, 8.68kB/s]
c:\ProgramData\Miniconda3\envs\expt\lib\site-packages\huggingface_hub\file_download.py:123: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\hf-models. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Downloading: 100%|██████████| 642/642 [00:00<00:00, 643kB/s]
Downloading: 100%|██████████| 1.04M/1.

### Setup the data modules

In [4]:
from datautils import DialogDataModule, DialogBatcher


DPATH = r"..\data\persuasionforgood_corpus"

is_pursuader = True
batcher = DialogBatcher(tokenizer, "Convince people to donate to charity.", is_pursuader=is_pursuader)
dm = DialogDataModule(DPATH, batcher=batcher, batch_size=2)

# Prepare the model

In [6]:
from modelling import DialogAgent


ROLE = "persuader" if is_pursuader else "persuadee"
SAVE_PATH = f"../models/base/{ROLE}"
CKPT = ""

model = DialogAgent(
    MPATH,
    embedding_size=len(tokenizer)
)
print("Loaded fresh model")

Downloading: 100%|██████████| 863M/863M [01:38<00:00, 8.79MB/s] 


Loaded fresh model


### Setup Callbacks

In [ ]:
from pytorch_lightning.callbacks import(
    EarlyStopping,
    ModelCheckpoint,
    RichModelSummary
)

checkpoint_callback = ModelCheckpoint(
    SAVE_PATH,
    filename=f'baseagent-{ROLE}-{{epoch}}-{{val_loss:.2f}}',
    monitor='val_soft_loss',
    save_weights_only=True,
)
early_stop_callback = EarlyStopping(
    monitor="val_loss",
    min_delta=1e-4, patience=3,
    verbose=False,
    mode="min"
)

# Train the model

In [ ]:
from pytorch_lightning import Trainer, seed_everything


seed_everything(42, workers=True)

trainer = Trainer(
    max_epochs=-1,
    deterministic=True,
    accumulate_grad_batches=4,
    callbacks=[checkpoint_callback, early_stop_callback, RichModelSummary()],
    accelerator='gpu',
    log_every_n_steps=16
)

trainer.fit(model, dm)